# Import packages

In [1]:
import warnings
from dataclasses import InitVar, dataclass, field

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from entsoe import EntsoePandasClient

warnings.filterwarnings("ignore")

# Pull data

In [2]:
def fetchdata(star_date, end_date, country):
    client = EntsoePandasClient(api_key="444fc771-5d0f-499f-9328-90c05c459219")
    start = pd.Timestamp(star_date, tz="Europe/Brussels")
    end = pd.Timestamp(end_date, tz="Europe/Brussels")
    generation = client.query_generation(country, start=start, end=end)
    generation = generation.iloc[
        :, generation.columns.get_level_values(1) == "Actual Aggregated"
    ]
    generation.columns = generation.columns.droplevel(level=1)
    return generation

In [3]:
tag_dict_renew = {
    "Biomass": "Biomass",
    "Geothermal": "Geothermal",
    "Hydro_storage": "Hydro Pumped Storage",
    "Hydro": "Hydro Run-of-river and poundage",
    "Hydro_res": "Hydro Water Reservoir",
    "Other_renew": "Other renewable",
    "Solar": "Solar",
    "Wind_off": "Wind Offshore",
    "Wind_on": "Wind Onshore",
}

tag_dict_NON_renew = {
    "Lignite": "Fossil Brown coal/Lignite",
    "Gas": "Fossil Gas",
    "Coal": "Fossil Hard coal",
    "Oil": "Fossil Oil",
    "Nuclear": "Nuclear",
    "Other": "Other",
    "Waste": "Waste",
}
tag_dict = tag_dict_NON_renew | tag_dict_renew


def formatdata(generation):
    df = pd.DataFrame(
        index=generation.index,
        columns=tag_dict.keys(),
    )
    for k, v in tag_dict.items():
        if v in generation.columns:
            df[k] = generation[v]
        else:
            df[k] = 0.0
    df = df.fillna(0.0)
    df["Renewables"] = df[tag_dict_renew].sum(axis=1)
    df["NonRenewables"] = df[tag_dict_NON_renew].sum(axis=1)

    df["Total"] = df["NonRenewables"] + df["Renewables"]

    return df

In [4]:
co2_dict = {
    "Biomass": 230,
    "Lignite": 1104,
    "Gas": 602,
    "Coal": 1104,
    "Oil": 1125,
    "Geothermal": 38,
    "Hydro_storage": 859,
    "Hydro": 11,
    "Hydro_res": 11,
    "Nuclear": 5,
    "Other": 700,
    "Other_renew": 300,
    "Solar": 35,
    "Waste": 700,
    "Wind_off": 13,
    "Wind_on": 13,
}
def calcuate_carbon_intensity(generation_STROMZEITEN):
    df_c02 = pd.DataFrame()
    for k, v in co2_dict.items():
        df_c02[k] = (generation_STROMZEITEN[k] * 1e3 * v) / 1e6
    df_c02["Total"] = df_c02.sum(axis="columns")
    df_c02["Carbon Intensity"] = (df_c02["Total"] * 1e6 / (generation_STROMZEITEN["Total"] * 1e3))
    return df_c02
    

In [13]:
start = '20230115'
end = '20230116'
country = 'BE'
gen_ENTSO = fetchdata(start, end, country)
gen_STROMZEITEN = formatdata(gen_ENTSO)
gen_CO2 = calcuate_carbon_intensity(gen_STROMZEITEN)
gen_ENTSO.to_csv(f'../Data/{country}_gen_RAW{start}_{end}.csv', index_label='Datetime')
gen_STROMZEITEN.to_csv(f'../Data/{country}_gen{start}_{end}.csv')
gen_CO2.to_csv(f'../Data/{country}_C02{start}_{end}.csv')  

In [14]:
gen_ENTSO

,Biomass,Fossil Gas,Fossil Oil,Hydro Pumped Storage,Hydro Run-of-river and poundage,Nuclear,Solar,Waste,Wind Offshore,Wind Onshore
2023-01-15 00:00:00+01:00,155.0,406.0,0.0,NaN,6.0,4912.0,0.0,95.0,1969.0,1617.0
2023-01-15 01:00:00+01:00,118.0,398.0,0.0,NaN,6.0,4911.0,0.0,98.0,1930.0,1603.0
2023-01-15 02:00:00+01:00,116.0,401.0,0.0,NaN,6.0,4909.0,0.0,97.0,1946.0,1460.0
2023-01-15 03:00:00+01:00,117.0,399.0,0.0,NaN,6.0,4906.0,0.0,101.0,1960.0,1475.0
2023-01-15 04:00:00+01:00,119.0,399.0,0.0,NaN,6.0,4901.0,0.0,104.0,2068.0,1616.0
2023-01-15 05:00:00+01:00,121.0,401.0,0.0,NaN,6.0,4901.0,0.0,108.0,2053.0,1803.0
2023-01-15 06:00:00+01:00,121.0,404.0,0.0,NaN,6.0,4903.0,0.0,107.0,2088.0,1862.0
2023-01-15 07:00:00+01:00,120.0,412.0,0.0,77.0,6.0,4902.0,0.0,107.0,2099.0,1975.0
2023-01-15 08:00:00+01:00,119.0,412.0,0.0,26.0,6.0,4901.0,1.0,105.0,2106.0,1997.0
2023-01-15 09:00:00+01:00,119.0,432.0,0.0,42.0,6.0,4900.0,224.0,102.0,2081.0,2079.0


In [ ]:
gen_ENTSO

In [ ]:
gen_STROMZEITEN

In [ ]:
gen_CO2

In [ ]:
def fetchdata_load(star_date, end_date, country):
    client = EntsoePandasClient(api_key="444fc771-5d0f-499f-9328-90c05c459219")
    start = pd.Timestamp(star_date, tz="Europe/Brussels")
    end = pd.Timestamp(end_date, tz="Europe/Brussels")
    load = client.query_load(country, start=start, end=end)
    load.to_csv(f'ENTSOE_DATA/{country}_load_RAW{start}_{end}.csv')
    return load

def fetchdata_prices(star_date, end_date, country):
    client = EntsoePandasClient(api_key="444fc771-5d0f-499f-9328-90c05c459219")
    start = pd.Timestamp(star_date, tz="Europe/Brussels")
    end = pd.Timestamp(end_date, tz="Europe/Brussels")
    prices = client.query_day_ahead_prices(country, start=start, end=end)
    prices.to_csv(f'ENTSOE_DATA/{country}_prices_RAW{start}_{end}.csv')
    return prices

In [ ]:
start = '20230101'
end = '20230116'
countries = ['BE', 'FR', 'PL']

for country in countries:
    load = fetchdata_load(start, end, country)
    print("load data fetched for: ", country)
    load.to_csv(f'ENTSOE_DATA/{country}_load_RAW{start}_{end}.csv')
    prices = fetchdata_prices(start, end, country)
    prices.to_csv(f'ENTSOE_DATA/{country}_price_RAW{start}_{end}.csv')
    print("prices data fetched for: ", country)